# TPC1: Análise de dados: doença cardíaca

Bibliotecas importadas:

In [ ]:
from prettytable import PrettyTable
import matplotlib.pyplot as plt

1. Crie uma função que lê a informação do ficheiro para um modelo, previamente pensado em memória;

In [ ]:
def read_file():
    data = dict()
    data['D'] = []
    data['ND'] = []
    
    file = open("myheart.csv", "r")
    i = 0
    for line in file:
        if i>0:
            append = True
            dados_linha = line.strip().split(",")
            if len(dados_linha) >= 6:
                
                idade = dados_linha[0]
                if idade.isnumeric():
                    idade = int(idade)
                else:
                    append = False
                    
                sexo = dados_linha[1]
                if sexo != 'M' and sexo != 'F':
                    append = False

                tensao = dados_linha[2]
                if tensao.isnumeric():
                    tensao = int(tensao)
                else:
                    append = False
                    
                colestrol = dados_linha[3]
                if colestrol.isnumeric():
                    colestrol = int(colestrol)
                else:
                    append = False
                
                batimento = dados_linha[4]
                if batimento.isnumeric():
                    batimento = int(batimento)
                else:
                    append = False
                
                if dados_linha[5] == '1':
                    temDoenca = True
                elif dados_linha[5] == '0':
                    temDoenca = False
                else:
                    append = False
                    
                if append is True:
                    if temDoenca is True:
                        data['D'].append((idade,sexo,tensao,colestrol,batimento))
                    elif temDoenca is False:
                        data['ND'].append((idade,sexo,tensao,colestrol,batimento))                    
        i += 1
        
    file.close()
    return data

2. Pense num modelo para guardar uma distribuição;

O modelo utilizado para guardar cada distribuição será um dicionário. Como chave de cada dicionário serão utilizados os valores da distribuição em questão e o respetivo valor associado a cada chave do dicionário corresponderá à frequência do valor da chave na distribuição em causa.

3. Crie uma função que calcula a distribuição da doença por sexo;

In [ ]:
def dist_sexo(data):
    dist_sexo = dict()
    dist_sexo['Masculino'] = 0
    dist_sexo['Feminino'] = 0

    for tuplo in data:
        sexo = tuplo[1]
        if sexo == 'M':
            dist_sexo['Masculino'] += 1
        elif sexo == 'F':
            dist_sexo['Feminino'] += 1
        
    return dist_sexo

4. Crie uma função que calcula a distribuição da doença por escalões etários. Considere os seguintes escalões: [30-34], [35-39], [40-44], ...

In [ ]:
def dist_idade(data):
    dist_idade = dict()
    x1_idade_min = 30
    x1_idade_max = None

    for tuplo in data:
        idade = tuplo[0]
        int_idade = (idade//5*5, (idade//5+1)*5-1)
        if int_idade not in dist_idade:
            dist_idade[int_idade] = 1
        else:
            dist_idade[int_idade] += 1
        
        if int_idade[0] < x1_idade_min:
            x1_idade_min = int_idade[0]
        
        if x1_idade_max is None:
            x1_idade_max = int_idade[0]
        elif int_idade[0] > x1_idade_max:
            x1_idade_max = int_idade[0]
    
    if x1_idade_max is not None:
        i = x1_idade_min
        while i<=x1_idade_max:
            if (i,i+4) not in dist_idade:
                dist_idade[(i,i+4)] = 0
            i+=5
                
    return dict(sorted(dist_idade.items()))

 5. Crie uma função que calcula a distribuição da doença por níveis de colesterol. Considere um nível igual a um intervalo de 10 unidades, comece no limite inferior e crie os níveis necessários até abranger o limite superior;

In [ ]:
def colestrol_min_max(data):
    colestrol_min = None
    colestrol_max = None
    for tuplo in data:
        colestrol = tuplo[3]
        
        if colestrol_min is None:
            colestrol_min = colestrol
        elif colestrol < colestrol_min:
            colestrol_min = colestrol
        
        if colestrol_max is None:
            colestrol_max = colestrol
        elif colestrol > colestrol_max:
            colestrol_max = colestrol
            
    return (colestrol_min, colestrol_max)

def dist_colestrol(data):
    colestrol_min, colestrol_max = colestrol_min_max(data)
    dist_colestrol = dict()
    
    i=colestrol_min
    while i<=colestrol_max:
        dist_colestrol[(i,i+9)] = 0
        i+=10
    
    for tuplo in data:
        colestrol = tuplo[3]
        int_colestrol = (colestrol_min+(colestrol-colestrol_min)//10*10, colestrol_min+((colestrol-colestrol_min)//10+1)*10-1)
        if int_colestrol not in dist_colestrol:
            dist_colestrol[int_colestrol] = 1
        else:
            dist_colestrol[int_colestrol] += 1
                
    return dict(sorted(dist_colestrol.items()))

 6. Crie uma função que imprime na forma de uma tabela uma distribuição;

In [ ]:
def print_distribuicao(table_title, table_fields, data):
    table = PrettyTable()
    table.title = table_title
    table.field_names = table_fields
    
    total = sum(data.values())
    for key in data:
        percentage = data[key] / total * 100
        table.add_row([key, data[key], '{:.2f} %'.format(percentage)])

    print(table)

def tuplo_para_intervalo(t):
    return f"[{t[0]}-{t[1]}]"

def print_distribuicao_int(table_title, table_fields, data):
    table = PrettyTable()
    table.title = table_title
    table.field_names = table_fields

    total = sum(data.values())
    for key in data:
        percentage = data[key] / total * 100
        table.add_row([tuplo_para_intervalo(key), data[key], '{:.2f} %'.format(percentage)])

    print(table)

7. Especifique um programa que ao executar apresenta as tabelas correspondentes às distribuições pedidas;

In [ ]:
data = read_file()
dict_dist_sex = dist_sexo(data['D'])
dict_dist_idade = dist_idade(data['D'])
dict_dist_colestrol = dist_colestrol(data['D'])

print_distribuicao("Distribuição da doença por sexo", ["Sexo", "Frequência", "Percentagem (%)"], dict_dist_sex)
print("\n", end='')
print_distribuicao_int("Distribuição da doença por escalão etário", ["Idade", "Frequência", "Percentagem (%)"], dict_dist_idade)
print("\n", end='')
print_distribuicao_int("Distribuição da doença por nível de colestrol", ["Colestrol", "Frequência", "Percentagem (%)"], dict_dist_colestrol)
print("\n", end='')

8. Extra: explore o módulo matplotlib e crie gráficos para as suas distribuições.

In [ ]:
def grafico_barras(titulo, eixo_x, eixo_y, data):
    x_values = [str(key) for key in data.keys()]
    y_values = data.values()

    barras = plt.bar(x_values, y_values)
    plt.bar_label(barras, labels=y_values)

    manager = plt.get_current_fig_manager()
    manager.set_window_title(titulo)

    plt.title(titulo)
    plt.xlabel(eixo_x)
    plt.ylabel(eixo_y)
    
    plt.show()
    
def grafico_barras_int(titulo, eixo_x, eixo_y, data):
    x_values = [tuplo_para_intervalo(key) for key in data.keys()]
    y_values = data.values()   

    barras = plt.bar(x_values, y_values)
    plt.bar_label(barras, labels=y_values)
    
    manager = plt.get_current_fig_manager()
    manager.set_window_title(titulo)

    plt.title(titulo)
    plt.xlabel(eixo_x)
    plt.ylabel(eixo_y)
    plt.xticks(rotation=45)
    
    plt.show()
    
def grafico_circular(titulo, data):
    tuplos_filtrados = [(key,value) for (key,value) in zip(data.keys(),data.values()) if value > 0]
    filtered_labels = [x[0] for x in tuplos_filtrados]
    filtered_values = [x[1] for x in tuplos_filtrados]
    
    plt.pie(filtered_values, labels=filtered_labels, autopct='%.2f %%')
    
    total = sum(filtered_values)
    plt.legend(
        labels=[('%s, %.2f %%') % (label, (value/total)*100) for (label,value) in tuplos_filtrados],
        prop={'size': 11},
        loc='upper left', 
        bbox_to_anchor=(-0.4, 1.03)
    )

    manager = plt.get_current_fig_manager()
    manager.set_window_title(titulo)   
    plt.title(titulo)

    plt.show()
    
def grafico_circular_int(titulo, data):
    tuplos_filtrados = [(tuplo_para_intervalo(key),value) for (key,value) in zip(data.keys(),data.values()) if value > 0]
    filtered_labels = [x[0] for x in tuplos_filtrados]
    filtered_values = [x[1] for x in tuplos_filtrados]
    
    plt.pie(filtered_values, labels=filtered_labels, autopct='%.2f %%')
    
    total = sum(filtered_values)
    plt.legend(
        labels=[('%s, %.2f %%') % (label, (value/total)*100) for (label,value) in tuplos_filtrados],
        prop={'size': 11},
        loc='upper left', 
        bbox_to_anchor=(-0.4, 1.03)
    )
        
    manager = plt.get_current_fig_manager()
    manager.set_window_title(titulo)
    plt.title(titulo)
    
    plt.show()

9. Extra: Menu para seleção de cada um dos gráficos implementados para visualização das distribuições pedidas acima. Para uma utilização mais intuitiva, recomenda-se a utilização do ficheiro 'TPC1.py'.

In [ ]:
def menu(dict_dist_sex, dict_dist_idade, dict_dist_colestrol):
    table = PrettyTable()
    table.title = "Menu de Gráficos"
    table.field_names = ["Nº", "Opção"]
    table.align['Opção'] = 'l'
    
    table.add_row(["", "Gráfico de Barras"])
    table.add_row(["---", "-------------------------------------------------------"])
    table.add_row(["1", "Distribuição da doença por sexo"])
    table.add_row(["2", "Distribuição da doença por escalão etário"])
    table.add_row(["3", "Distribuição da doença por nível de colestrol"])
    table.add_row(["---", "-------------------------------------------------------"])
    table.add_row(["", "Gráfico Circular"])
    table.add_row(["---", "-------------------------------------------------------"])    
    table.add_row(["4", "Distribuição da doença por sexo"])
    table.add_row(["5", "Distribuição da doença por escalão etário"])
    table.add_row(["6", "Distribuição da doença por nível de colestrol"])
    table.add_row(["---", "-------------------------------------------------------"])
    table.add_row(["0", "Sair"])
    
    saida = -1
    opcao_invalida = False
    while saida != 0:        
        if not opcao_invalida:
            print("\n", end='')
            print(table)

        option_flag = False
        opcao = input("\nIntroduza a sua opção: ")
        while not option_flag:
            try:
                opcao = int(opcao)
                if opcao < 0 or opcao > 6:
                    print("Opção inválida!")
                    option_flag = False
                    opcao = input("\nIntroduza novamente a sua opção: ")
                else:
                    option_flag = True
            except ValueError:
                print("Opção inválida!")
                option_flag = False
                opcao = input("\nIntroduza novamente a sua opção: ")

        saida = opcao
        opcao_invalida = False
        
        if saida == 1:
            grafico_barras("Distribuição da doença por sexo", "Sexo", "Frequência", dict_dist_sex)

        elif saida == 2:
            grafico_barras_int("Distribuição da doença por escalão etário", "Idade", "Frequência", dict_dist_idade)

        elif saida == 3:
            grafico_barras_int("Distribuição da doença por nível de colestrol", "Colestrol", "Frequência", dict_dist_colestrol)

        elif saida == 4:
            grafico_circular("Distribuição da doença por sexo", dict_dist_sex)

        elif saida == 5:
            grafico_circular_int("Distribuição da doença por escalão etário", dict_dist_idade)

        elif saida == 6:
            grafico_circular_int("Distribuição da doença por nível de colestrol", dict_dist_colestrol)

        elif saida != 0:
            print("Opção inválida!")
            opcao_invalida = True

menu(dict_dist_sex, dict_dist_idade, dict_dist_colestrol)